# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [8]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [9]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [10]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#




last_height = browser.execute_script("return scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    print('網頁更新中...')
    
    new_height = browser.execute_script("return document.body.scrollHeight")    
    if new_height == last_height:
        print('到達頁面底端')
        break 
    last_height = new_height
        
        
        

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [11]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [12]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 銅板價買股票　專家曝這兩個族群基期低又具成長性

圖、文／鏡週刊
銅板股由於價格便宜，受到許多小資族青睞，股市專家孫慶龍認為，不論是高價股或是低價股，能不能買它的關鍵仍在基本面與未來成長性。他認為，目前市場上有二個族群，不但基期偏低，同時具備成長性。
股價會低，多半原因是所屬產業面臨到比較大的競爭狀態，或是產業看不到未來成長性，無法受到市場認。，其實不難發現，高股價的公司，產業趨勢通常較為明顯。」孫慶龍認為，長期來看，50元以下的股票若要發展成百元以上的中、高價股，關鍵因素就在所屬產業的未來成長性。「許多市場大戶的長線資金，也都相當喜歡布局低價股。」
而他進一步透露，目前有二個產業目前仍處於基期低，且未來成長性顯著。其一是太陽能。「太陽能產業未來20年將成為人類最主要的發電方式。」孫慶龍表示，太陽能是真正取之不盡的能源，國外研究機構更指出，太陽光照射到地球表面1個小時，極端假設若人類能夠全部接收，100％轉換成電能，便可供全人類使用1年。
目前太陽能占全球發電量僅不到1％，若技術持續進步，市場樂觀預估往後10年，太陽能廠至少有3成的成長空間。孫慶龍指出，台廠積極開發太陽能技術的公司茂迪，8月漲勢凌厲，從原本不到10元價格，一路漲到20塊，主要原因正是來自其太陽能的轉換效能，出現突破性的成長。「同時，觀察技術面，該檔股票價格離10年線還非常遠，基期非常低。」
「茂迪第二季營收比去年第四季還衰退，但獲利卻從負轉正。」孫慶龍指出，據傳茂迪開發的太陽能板，在同樣的固定面積下，轉換效能從過去平均的18％，提升至21％。「別小看這一點點的提升，事實上是非常大的進步。」另外，由於台股市場中，純太陽能公司少，若投資人想要布局，可考慮選擇太陽能相關ETF。
至於另一個具備產業成長性的族群，孫慶龍認為是營造建設公司。他提到，主要來自兩面向：一、台商回流；二、政府及企業陸續開始擴大投資。這些都會帶動營建業及房子的需求。而他相對看好目前基期偏低、股價不到20元的欣陸控股。
欣陸控股除了新建案會在近二年陸續完工，第三家支撐集團營運的子公司，為負責地下化汙水環境工程的欣達環工，目前佔營收比重16％，隨著綠能與環保意識抬頭，欣達環工近年營收表現持

▲凱基證券傳出有營業員受不了業績壓力，投入自己的資金卻慘賠。（圖／CTWANT提供）
圖文／CTWANT
許多人對於金融業都有憧憬，認為整天坐在辦公室薪水又很高，不過其中的壓力僅有身在其中才能體會，近日券商及營業員間瘋傳一訊息，指1名證券業的L姓營業員因為達不到業績目標，投入自己的資金卻慘賠，最後選擇以激烈的方式來控訴。
1名在凱基證券不到四十歲的L姓營業員，因為達不到主管要求的業績，就將自己的資金投入操作當沖，原以為只要選擇好的標的，每天只要小賺就好，但沒想到卻遭受巨大虧損。
就在8月19日民俗月鬼門開當天，L姓營業員與主管在辦公室裡發生激烈爭執後，最後選擇穿紅衣墜河身亡，留下的遺書卻是指名給經理人，而此消息也在營業員間瘋傳。
有員工私下透露，主管告訴他們的是L姓營業員是因為在外面欠債太多，再加上已經罹患憂鬱症許久，才會發生憾事。
許多網友都在社群中討論，表示一般客戶只要虧損20萬，證券公司的營業員就會致電關心，而L姓營業員虧損的情形，難道公司都沒發現；另名網友則表示，有陣子虧了許多錢，營業員打電話來道德勸說，希望能休息一陣子，但卻聽到電話那頭營業員的主管說「客戶不在你這輸錢，也會去別的地方輸錢」。
事實上，類似營業員的自殺案件在2012年證所稅上路後，就時常傳出營業員業績量沒到或者自己投入股海慘賠後，雖然有零星案件爆發出來，但更多的卻是被公司壓下。
►鍋寶養生燉鍋原價4760元「限時下殺890元」
----------------------------------------------------------------------

[11] 台鈴與Gogoro Solutions合作　開發全新電動車品牌 eReady下線典禮

▲台鈴與Gogoro Solutions合作，開發全新電動車品牌。（圖／市府提供）
記者楊淑媛／桃園報導
桃園市長鄭文燦28日前往平鎮區、參訪台鈴工業表示，桃園是汽機車工業的基地，也是電動車的搖籃，台鈴與同樣是桃園電動車大廠的Gogoro Solutions合作開發全新電動車品牌 eReady，為機車產業帶來全新的技術；而桃園在電動車的領域目前是全台灣第一，在基礎設施上，包含充電站、換電站、公共租賃等系統都非常完備。
▲台隆工業公司董事長黃教漳表示，台鈴工業是台灣首家將沙灘車外銷到美國的製造商。▼全新電動車品牌 eReady下線典

▲豪華的飯店大多價格不斐。（圖／取自Pexels）
記者曾筠淇／綜合報導
不少人都熱衷於旅遊，為了把出遊經費全都花在食物和玩樂上，出門前通常會先精打細算，找尋價格實惠、CP值高的飯店或旅館。一名網友日前收到知名飯店、酒店優惠價的通知，豈料原本價格幾乎都破萬元，打折後的價碼才親民許多，而他就好奇，「原價破萬的飯店都是誰在住啊？」
原PO表示，這些飯店、酒店的原價幾乎都破萬元，且有幾間更是一晚就要價將近2.5萬元，如果沒有打折的話，CP值似乎不太高，也不太值得特別去住，「這麼貴都沒有打折的話，到底都是誰在住啊？」
▲不少飯店的原價都要破萬元。（圖／翻攝自PTT）
貼文一出，不少網友紛紛留言回應，「出差報公帳的人」、「高層出差」、「有錢人比你想像的多」、「婚宴附的房也是有啦」、「用國旅卡的搶著住，就是這樣價格才居高不下」、「出差阿，跟企業有簽約，員工住都白菜價」。
另外也有網友點出其中奧妙，「原價而已，實際訂房根本沒有」、「定價只是拿來殺價用的」、「弄個優惠讓消費者好像賺到一樣」、「打折好看吧」。
----------------------------------------------------------------------

[24] 加油50元就能抽機車！憑9月發票不需登錄...台灣中油10/19公布

▲台灣中油今（28）日宣佈9月將推出抽獎活動。（圖／台灣中油網站）
圖文／CTWANT
為響應政府推動振興經濟，台灣中油公司自9月1日至9月30日推出「防疫新生活，加油去旅遊」抽獎活動，獎項包含百輛機車、國內離島來回機票、台鐵環島列車套票等，消費者只要在活動期間到台灣中油體系加油站單筆交易發票金額50元（含）以上，憑發票不需登錄，就能享有抽獎機會。
台灣中油為防疫新生活及振興經濟共襄盛舉盡一份力，9月再推出以「行．旅」為主題的抽獎活動，獎項包含了百輛光陽機車、台鐵「環島之星」觀光列車旅遊套票及國內華信、立榮航空離島（澎湖、金門）來回機票等，合計金額約千萬元。
台灣中油表示，消費者只要在活動期間到台灣中油體系加油站交易（含加油、洗車及商店購物等），交易發票金額50元（含）以上皆有抽獎機會，不需要登錄；台灣中油抽獎後預計在10月19日公布中獎發票，幸運得主可就近至指定直銷櫃台或掛號郵寄活動專用信箱辦理兌獎手續。
台灣中油指出，幸運得獎者將可騎著機車、搭乘台鐵環

文／洪寶山
第二季財報公布近尾聲，台積電與聯發科為首的半導體是贏家，疫情下的航班大減，仰賴海運，長榮為首的航運業也意外繳出好成績，遠距辦公教學興起，原以為是一時，但防疫中心指揮官陳時中接受訪問時表示，台灣要恢復疫情前的正常生活還要一年，等待有效的疫苗或特效藥問世，才能結束疫情。換言之，是時候重新調整心態面對疫情改變了世界，特別是就業市場。
美國勞工部勞動統計局(BLS)於八月七日指出，二○二○年六月永久性失業人數為288.3萬人，是二○一四年五月以來新高。二○二○年七月美國失業十五至二十六周的待業族暴增241%至648.4萬人，創一九四八年開始統計以來最高，長期失業(失業時間至少達二十七周)人數增加11萬至150.1萬，創二○一七年十二月以來最高。
換言之，二○二○年七月美國失業至少十五周的人數達到798.5萬人。對照當月份1.5987億的勞動人口，這意味著中長期失業率達5%。
散戶熱情追捧科技股
而這部分的人將來很有可能在疫情結束後重返老本行，因為這些崗位隨著疫情的衝擊，業主透過互聯網或科技自動化替代了人力，成為永久性的失業，如果沒有透過政府的職業訓練轉行，若不是永久退出就業市場，便是投入了股市當韭菜。
所以當蘋果(AAPL)宣布一拆四、特斯拉(Tesla)一拆五，目標就是鎖定散戶族群成為推升股價的新動力，果然八月二十一日蘋果與特斯拉股價雙雙創499.47美元、2095.49美元歷史新高，成為這次疫情的贏家。
這批因疫情失業而投入股市的散戶們，拿著政府給的失業金投入股市，果然有如初生之犢不畏虎，熱情追捧科技股，FAANG、微軟、英特爾、超微、英偉達、高通等大廠的股價都強勢創新高，那斯達克指數也領先道瓊指數頻創歷史新高，暗示著透過科技創新，世界經濟得以持續運作，不至於因入出境隔離十四天的限制而停擺，縱使因社交距離而導致實體行業的需求衰退，但虛擬的電商與社交平台都成為替代方案，下半年的投資決勝點，便是從半年報找出轉機成長股。
持續提高技術門檻　維持競爭力
IC測試載板廠雍智科技的EPS在二○一七年見到4.39元後，觸底反彈，主要是受惠於中國大陸的5G自二○一八年開始佈建基地台，EPS上升到5.94元，二○一九年5G高階手機問世，EPS進一步提升到7.85元，二○二○年中階手機上市，聯發科在中國大陸5G市占率上升，帶動雍智5G和射頻晶片相關IC測試載板業績成長。
此外，

▲屏東總圖今天正式啟用，位於千禧公園內被稱為「森林系圖書館」。（圖／記者陳建宇攝）
記者陳建宇／屏東報導
屏東縣立圖書總館今天正式開幕，因為座落在千禧公園內，被喻為是「森林系」圖書館，而圖書館的落成也強化周邊民眾的休閒功能，附近房仲業者表示對環境機能具有加分效果，而周邊的中古華廈房價也很親民，最低每坪9萬元就能入手。
今天開幕的屏東縣立圖書總館其實原本就存在，重新整建後也擴大樓地板面積達到3179坪，位於屏東市知名的千禧公園內，周邊花木扶疏，被喻為是「森林系」的圖書館，更加入演講廳空間，讓圖書館變成更多功能的場所。
▲廣東路是屏東總圖周邊重要道路，沿線商家相當多。（圖／記者陳建宇攝）
而屏東縣立圖書總館所在的千禧公園是屏東市2大公園之一，與中山公園齊名，地理位置也在屏東市區，因此附近的生活機能非常豐富。住商不動產屏東明正加盟店總經理楊信華表示，包圍千禧公園的路段如廣東路、自由路、大連路和勝利路等，都是屏東的主要道路，沿線商業行為興盛，加上千禧公園、國民運動中心以及剛啟用的圖書總館，生活機能非常便利。
不過這樣便利的區段，房價其實並不高，東森房屋屏東家樂福店經理魏漢傑表示，因為千禧公園周邊屬於發展較早的區段，因此開發非常成熟，沒有多餘的土地可蓋新建案，中古屋屋齡普遍較高，華廈產品有不少屋齡都在20年以上，最便宜每坪9萬多元就買得到，若是屋況較好、有整理、附帶車位的產品，價格則有機會在每坪11萬元以上。
楊信華表示，若離千禧公園稍遠一點則有屋齡新一點、約10多年的華廈，房價每坪約12~13萬元，10年內的物件則可以到每坪15~16萬元。透天厝也是千禧公園附近的主力產品，魏漢傑表示屋齡20年以上的舊透天若是在6米巷內，總價約500萬元就能買到，而臨8~10米路的產品總價則在900~1100萬元。
----------------------------------------------------------------------

[40] 國有非公用土地改良利用案件　創造1286億元商機

▲滬尾藝文休閒園區。（圖／國產署提供）
記者吳靜君／台北報導
財政部國產署與目的事業主管機關共同推動國有非公用土地改良利用業務，引進多種產業類型包含產業園區、商業設施、觀光休憩、再生能源等，可同時兼顧土地活化及產業發展需求。
根據國產署統計，至今年7月底止，已經簽約且存續中有6

----------------------------------------------------------------------

[51] 亞太經合會科技與創新論壇登場　宏碁董座陳俊聖分享台灣防疫成果

▲防疫在家工作導致菲律賓民眾排隊搶購電腦。（圖／宏碁提供）
記者姚惠茹／台北報導
宏碁（2353）董事長陳俊聖受ABAC(APEC Business Advisory Council，亞太經合會企業諮詢委員會)主席Rohana Mahmood之邀，針對疫情下如何促進亞太經濟復甦及未來發展，提供各經濟體科技及經濟相關政府單位建言，同時分享台灣如何運用科技防疫。
陳俊聖代表ABAC及台灣參加亞太經合會架構下科技政策與科技交流第十六次「科技與創新政策夥伴關係」論壇會議（16th APEC Policy Partnership on Science, Technology and Innovation Meeting），主題為「Laying the Groundwork for Economic Recovery and Building Resilience」。
陳俊聖表示，短期的要務是保持市場的開放性，讓經濟活動可以持續，並致力恢復供應鏈，中小型及微型企業因資源弱勢，所受到的衝擊最大，建議政府單位應有完善財政措施協助其度過難關，而這百年罕見的疫情也徹底改變人類的生活，加速線上活動和線上交易的成長。
陳俊聖指出，更需具備資訊安全意識，企業也應貢獻己力加速各行各業數位轉型；中長期而言，各經濟體應大力支持世貿組織（WTO），促進自由、多邊貿易的角色，對開放邊境達成共識，並積極研商開放準則，以促進旅遊及商務活動，重建亞太經濟。
陳俊聖分享，台灣在疫情的初期即採用口罩管制配給，甚至開發口罩地圖app，透明的資訊能有效降低民眾的恐慌，而宏碁協助台灣疾病管制署（CDC）建立醫院感染管制即時通報系統，隨時掌握傳染病感染源，讓醫院政策制定者可調整防制傳染病策略及資源調整的行動。
陳俊聖說明，宏碁利用人工智慧技術在智慧醫療上多有著墨，如檢測糖尿病視網膜病變，迅速篩選出潛在病患供醫師確認；透過模型建立和數據分析，流感感染擴散的預測系統也是防疫官員決策的利器；目前正建立以自然語言處理技術平台分析各國新聞，掌握全球傳染病的源起及演進。
後續座談中針對澳洲產業、科學、能源及資源部官員

▲在弱勢美元、低利率等4大因素支撐下，星展銀行看好金價年底前將走揚。（示意圖／路透）
記者紀佳妘／台北報導
全球疫情尚未緩解，推升投資人避險情緒高漲，金價衝上歷史新高後，近期呈現回檔整理，不過星展銀行認為，在量化寬鬆、弱勢美元、低利率以及全球不確定性上升的環境下，今年剩餘時間內金價仍將走揚。
星展銀行表示，對全球財政刺激和量化寬鬆（QE）規模的擔憂繼續使投資人懷疑法定貨幣的可信度；由於中央銀行和投資人尋找避風港，黃金作為替代貨幣也大幅提升了吸引力。
隨著美元的走弱，金價日前升至每盎司2,000美元以上的歷史新高後，近期呈現回檔整理。不過，星展銀行認為，寬鬆貨幣政策、負實際利率以及投資人需求上升，將支撐金價未來12個月升至每盎司2,000美元以上。
此外，黃金價格看漲將有利於黃金礦商，星展銀行表示，今年以來，黃金平均價格為1,695美元，而去年的平均價格為1,393美元，意味著礦商的利潤率將大幅增長，顯示黃金礦業公司扮演槓桿角色，金價上漲將提升其獲利，產量增加也提升其利潤率。
今年初受到疫情衝擊，全球經濟面臨有史以來最嚴重的衰退，但星展銀行認為，在經濟和公司利潤萎縮之際，金礦公司成為獲利企業，但評價面卻被低估的產業之一。
星展銀行預估，金礦公司表現將優於金價表現，主要是股價漲幅通常超越金價的漲幅，且金礦公司會支付股息。今年以來，紐約證交所Arca黃金礦業指數（GDM）上漲了37%，表現優於金價的28%漲幅。
----------------------------------------------------------------------

[65] 衝動入手40年套房！她怕「老屋越老越麻煩」賣不掉　網一看地點：穩了

記者黃可昀／綜合報導
屋齡高的物件最讓人擔心的就是維修問題，漏水、壁癌、管線老化等問題修起來也是筆龐大費用。一名女子透露，家人最近衝動入手北市40年華廈套房，本來想當作度假屋，但做了功課後才發現，「老屋越老越麻煩」，擔心住沒多久屋況問題就會一次湧現，於是PO文問大家該如何維護？
原PO在臉書社團「買房知識家（Q你的A）」表示，這間套房並非位在蛋黃區，目前出租中，房客也有打算繼續租下去，於是家人暫時打消當作度假屋的念頭，轉而把這間房當作投資物件，所以仔細地做了功課，才驚覺屋齡是購屋時重要的參考條件之一，40年已經算屋齡很高，再過幾年可能就要花錢整

「新潤心城」擁抱土城最精華市區，坐鎮金城路、裕民路核心，長庚醫院、土城清水交流道等百億重大建設環繞在側，掌握板南線海山站、萬大線LG10站、三鶯線燙金三捷，牽手三大商圈、秀泰影城、日月光百貨、斬龍山公園、運動中心，10分鐘步行距離，食衣住行ALL IN ONE機能滿檔，如今推出限量壓軸小坪數，吸引購屋民眾目光，以低總價輕鬆入主土城核心門戶，後勢增值潛力無窮。
----------------------------------------------------------------------

[76] 必翔實業遭掏空　董座伍必翔與前妻蔣清明母女遭起訴

▲必翔實業董事長伍必翔涉掏空公司遭起訴。（圖／資料照）
記者劉昌松／台北報導
電動車大廠必翔實業董事市長伍必翔，涉嫌與前妻、前董座蔣清明，分別為了衝高子公司業績並護盤，造成必翔損失約4億元，又挪用大陸子公司人民幣約4億元(約新台幣17億元)，以假交易、借貸等名目頻繁匯款，部分最後流入蔣的女兒、陽明公司副總戴佳瑀個人戶頭，北檢今天依違反《證交法》特別背信、非常規交易等罪起訴伍、蔣、戴等6人。
現已下市的必翔實業前董事長蔣清明，因為在2014年到2016年間，因得知必翔在海外面臨巨額求償訴訟，恐導致股價下跌、影響公司質押股票價值等因素，聯合炒手挪用公司資金，拉抬股價，台北地院一審在2019年2月依操縱股價高買證券罪，判蔣徒刑13年，並計算出蔣的全部不法犯罪所得超過8億元。
蔣遭重判案件目前上訴二審中，檢調單位持續追查，發現蔣另外在2016年起，多次挪用在大陸的子公司陽明公司，用假交易或借貸等名義，頻匯款總計人民幣4億元給其他公司，對方再陸續「歸還」款項，其中部分流入陽明公司副總、也是蔣的女兒戴佳瑀戶頭。
到2017年間，必翔實業轉投資的必翔電能財報被會計師發現有問題，部分應收帳預期，應該調整變成虧損，當時的董事長伍必翔和蔣清明為了確保必翔電能也能上市，挪用必翔資金大買必翔電能股票，希望拉抬股價，結果必翔電能財報公告後，必翔電能股價大跌，造成必翔遭挪用的資金損失4億元。
療癒！紓壓！心訣大公開
----------------------------------------------------------------------

[77] 償債！JDI手機螢幕工廠　3.9億美元賣給夏普

▲日本顯示器公司（J

記者王致凱／綜合報導
美國聯準會（Fed）主席鮑爾（Jerome Powell）27日在全球央行年會上表示，聯準會已經調整政策，讓通貨膨脹率在一段時間內平均值落在2％，意味著聯準會容許通膨暫時高於2％，讓美國低利率政策維持更長一段時間。受消息激勵，美股周四早盤走高，標普500指數首次站上3500點大關。
鮑爾表示，聯準會已經同意採取「平均通膨目標」，允許通膨率維持在2％以上「一段時間」。聯準會過去都將通膨率2％視為經濟健康的指標，這也代表聯准會捨棄了失業率降低至一定水準，將使通膨提高至危險水準的看法。
未來即使美國失業率下滑，聯准會也更可能傾向不升息，將低利率維持更長一段時間。鮑爾說，聯准會將會用盡一切辦法來支撐經濟。
道瓊工業平均指數早盤上漲254點或0.9％，已經抵銷道指30檔成分股2020年的平均跌幅。標普500指數上漲0.6％，史上首次盤中突破3500點，那斯達克綜合指數也小幅上漲。
►「粉漾快煮壺」輸入hercp103再折103元，一個只要394元
----------------------------------------------------------------------

[88] TikTok出售案預計「數天內宣布」　美媒：規模200億至300億美元

記者王致凱／綜合報導
抖音國際版TikTok執行長梅耶爾（Kevin Mayer）27日請辭後，有消息人士向媒體透露，TikTok出售交易案已經接近達成，預期未來幾天內就可能宣佈，目前賣方仍在考慮三家美國企業的出價。
根據CNBC報導，一名不願透露姓名的人士表示，梅耶爾的辭職消息原本計畫下周與出售協議一同宣布，但他決定稍微提前了宣布時間點。梅耶爾今年初才剛從迪士尼跳槽至TikTok。
根據TikTok公司寄給員工的內部備忘錄，美國業務主管帕帕斯（Vanessa Pappas）將暫代執行長職務。
至於TikTok出售交易案，有兩名知情人士表示，TikTok還在考慮甲骨文（Oracle）、微軟（Microsoft）和第三家美國企業的報價。知情人士透露，沃爾瑪（Walmart）一直與軟體銀行（SoftBank）合作推動此一潛在交易案；沃爾瑪稍後證實考慮收購TikTok，並且將與微軟合作。
對此，包括TikTok和軟銀和沃爾瑪公司都沒有發表評論。知情人士表示，TikTok可能出售其在美國、加拿大